# Battle of the Neighborhoods

- Coursera Capstone Project

This notebook foucses on collecting and preparing the required data for exploring the neighborhoods in Metropolitan Melbourne. 

### 1. Introduction

A start-up company plans to expand and grow its business in **Australia** in the next five years. Its founders are looking to establish a headquarters in  `Melbourne, Victoria`.  Furthermore, the company needs to provide a sustainable, comfortable, and safe work-life environment for their senior-level employees who will travel to Australia and supervise the business establishment. This project will use machine learning techniques to anylze the characteristics for the neighbourhoods (suburbs) in Melbourne regarding *crimes*, *venues*, and *rental prices* to determine the best possible suburbs to meet their requirements. At last, a list of recommended neighbourhoods (suburbs) with a full report will be submitted to the company based on the analysis of the clustering results.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
import urllib.parse
import os

DATASETS_PATH = './datasets'

### 2. Collect and Preprocess Dataset

In this section, the main objective is to collect and prepare four datasets namely `geographical` dataset, `crime record` dataset, `rental` dataset, and `venue` dataset. In the later work, the datasets are implemented in conjunction with machine leaning models to explore the characteristics of suburbs in Metropolitan Melbourne. Accordingly, the results are used to determine the suitable suburbs for the client which meet their requirements. 


#### Metropolitan Melbourne Geographical Dataset

In this section, the goal is to collect the geographical data of Metropolitan Melbourne.

Objective 1: accquire `name`, `postcodes`, and `councils` for suburbs in Metropolitan Melbourne based on the information in reference(https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs)

Objective 2: collect `latitude` and `longitude` for each suburb using Nominatim API

In [2]:
MELBOURNE_URL = 'https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs'
html_content = requests.get(MELBOURNE_URL).text
soup_melbourne_suburb = BeautifulSoup(html_content, 'html.parser')

In [3]:
# define a function that extract and clean the information in MELBOURNE_URL 
def htmlscraper(soup):
    html = soup.find_all(['h3','ul'])
    melbourne_data = []
    council=[]
    for item in html:
        if item.name == 'h3':
            council = re.findall(r'\b[A-Z]\w+? of \b[A-Z,a-z, ]*',item.get_text())
            if council:
                pocket_council = council

        elif item.name == 'ul' and council:
            html_lis = item.find_all('li')
            for i in range(len(html_lis)):
                temp = html_lis[i].get_text()
                suburb = re.findall(r'\b[A-Z,a-z, ]* \d{4}', temp)
                if suburb and len(suburb[0]) < 40:
                    suburb = suburb[0].split(' ')
                    suburb_name = ''
                    for j in range(len(suburb)-1):
                        suburb_name += suburb[j]

                        if j == (len(suburb)-2):
                            break

                        suburb_name += ' '
                    temp_list = [suburb_name,suburb[-1]]
                    temp_list.append(pocket_council[0])
                    melbourne_data.append(temp_list)   
    return melbourne_data

In [4]:
melbourne_data = htmlscraper(soup_melbourne_suburb)

In [5]:
# create a dataframe for melbourne suburbs
melbourne_df = pd.DataFrame(melbourne_data,columns=['Suburb','Postcode','Council'])
melbourne_df.head()

,Suburb,Postcode,Council
0,Carlton,3053,City of Melbourne
1,Carlton North,3054,City of Melbourne
2,Docklands,3008,City of Melbourne
3,East Melbourne,3002,City of Melbourne
4,Flemington,3031,City of Melbourne


In [6]:
melbourne_df.shape

(585, 3)

In [7]:
# sort the dataset based on postcodes
melbourne_df = melbourne_df.sort_values('Postcode').reset_index(drop=True)
melbourne_df.head()

,Suburb,Postcode,Council
0,October,2007,City of Wyndham
1,Use dmy dates from October,2012,City of Wyndham
2,Melbourne,3000,City of Melbourne
3,East Melbourne,3002,City of Melbourne
4,West Melbourne,3003,City of Melbourne


In [8]:
# delete the first two rows in the dataset 
melbourne_df = melbourne_df.drop(index=[0,1]).reset_index(drop=True)
melbourne_df.head()

,Suburb,Postcode,Council
0,Melbourne,3000,City of Melbourne
1,East Melbourne,3002,City of Melbourne
2,West Melbourne,3003,City of Melbourne
3,Melbourne,3004,City of Melbourne
4,Melbourne,3004,City of Port Phillip


In [9]:
melbourne_df.shape

(583, 3)

In [10]:
# check the info of dataset
melbourne_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Suburb    583 non-null    object
 1   Postcode  583 non-null    object
 2   Council   583 non-null    object
dtypes: object(3)
memory usage: 13.8+ KB


In [11]:
# change the data type of postcode for later work
melbourne_df['Postcode'] = melbourne_df['Postcode'].astype('int64')
melbourne_df['Postcode'].dtype

dtype('int64')

In [12]:
missed_suburbs = [['Prahran', 3181, 'City of Stonnington'],
        ['Malvern', 3144, 'City of Stonnington'],
        ['Mont Albert North', 3129, 'City of Whitehorse'],
        ['Fraser Rise', 3336, 'City of Melton'],
        ['Deanside', 3336, 'City of Melton'],
        ['Taylors Hill', 3037, 'City of Melton'],
        ['Fieldstone', 3024, 'City of Melton'],
        ['Grangefields', 3335, 'City of Melton'],
        ['Harkness', 3337, 'City of Melton'],
        ['Strathtulloh', 3338, 'City of Melton'],
        ['Weir Views', 3338, 'City of Melton'],
        ['Thornhill Park', 3335, 'City of Melton'],
        ['Cobblebank', 3338, 'City of Melton'],
        ['Aintree', 3336, 'City of Melton']]

In [13]:
missed_suburbs_df = pd.DataFrame(missed_suburbs, columns=['Suburb', 'Postcode','Council'])

In [14]:
melbourne_df = melbourne_df.append(missed_suburbs_df, ignore_index=True)
melbourne_df.shape

(597, 3)

The preliminary study indicated that a suburb in Melbourne can be shared with multiple councils. Therefore, we need to check if there is any `duplicated entry` in Suburb with the same Postcode,

In [15]:
melbourne_df.Suburb.value_counts()

Bundoora            3
Tullamarine         3
Melbourne           3
Park Orchards       2
St Kilda East       2
                   ..
Roxburgh Park       1
Noble Park North    1
St Albans           1
Tremont             1
Shoreham            1
Name: Suburb, Length: 545, dtype: int64

In [16]:
# delete duplicates in dataset
melbourne_df_non_dup = melbourne_df.drop_duplicates(['Suburb','Postcode']).reset_index(drop=True)
melbourne_df_non_dup.Suburb.value_counts()

Melbourne      2
Maribyrnong    1
Warburton      1
Mickleham      1
Northcote      1
              ..
Deer Park      1
Moorooduc      1
Cremorne       1
Sandringham    1
Shoreham       1
Name: Suburb, Length: 545, dtype: int64

In [17]:
melbourne_df_non_dup.shape

(546, 3)

There are still two *Melbourne* in the dataset with different postcode. To avoid confusion in later work, the suburb's name with `Postcode == 3004` was changed from `Melbourne` to `St Kilda Rd`.

In [18]:
melbourne_df_non_dup.loc[melbourne_df_non_dup['Postcode'] == 3004, 'Suburb'] = 'St Kilda Rd'
melbourne_df_non_dup.loc[melbourne_df_non_dup['Postcode'] == 3004]

,Suburb,Postcode,Council
3,St Kilda Rd,3004,City of Melbourne


Adding Geo-coordinates into the dataset for each suburb. This will contribute to finding venues in each suburb using Foursquare API. We could also use these coordinates to calculate the travel distances for employees that access nearby veunes.

In [19]:
# create a function to find coordinates for suburbs based on the name and postcode using nominatim api

def locationfinder(data_df):
    lat = []
    lon = []
    for ix in range(data_df.shape[0]):
        suburb = data_df.loc[ix,'Suburb']
        postcode = data_df.loc[ix,'Postcode']
        address = '{}, Victoria, {}'.format(suburb,postcode)
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) + '?format=json'
        response = requests.get(url).json()
        lat.append(response[0]['lat'])
        lon.append(response[0]['lon'])
    return lat,lon

In [20]:
# call function to gather coordinates
mel_lat, mel_lon = locationfinder(melbourne_df_non_dup)
melbourne_df_non_dup['Latitude'] = mel_lat
melbourne_df_non_dup['Longitude'] = mel_lon

In [21]:
melbourne_df_non_dup.shape

(546, 5)

In [22]:
melbourne_df_non_dup.head(10)

,Suburb,Postcode,Council,Latitude,Longitude
0,Melbourne,3000,City of Melbourne,-37.8142176,144.9631608
1,East Melbourne,3002,City of Melbourne,-37.815588,144.98255534496568
2,West Melbourne,3003,City of Melbourne,-37.8106885,144.9204039725214
3,St Kilda Rd,3004,City of Melbourne,-37.8358215,144.9763639
4,South Wharf,3006,City of Port Phillip,-37.8253445,144.9523945305609
5,Southbank,3006,City of Port Phillip,-37.8253925,144.96357766652397
6,Docklands,3008,City of Melbourne,-37.8178695,144.93865434802785
7,Seddon,3011,City of Maribyrnong,-37.8067725,144.8915967
8,Footscray,3011,City of Maribyrnong,-37.801366,144.9024240873921
9,Kingsville,3012,City of Maribyrnong,-37.80884665601401,144.8553793430576


#### Criminal Incident Data of Metropolitan Melbourne

In this section, the main objective is to collect and clean the crime incident records of Metropolitan Melbourne. 

The reference listed below is a copy of Victoria Police Crime Statistics Report Data. Using the collected geographical data above, we can filter out the crime incidents recoreded in Metropolitan Melbourne. 

https://www.crimestatistics.vic.gov.au/sites/default/files/embridge_cache/emshare/original/public/users/202103/f1/092cf1840/Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2020.xlsx

In [23]:
# download the dataset and save it as a dataframe
CRIME_VIC_URL = 'https://www.crimestatistics.vic.gov.au/sites/default/files/embridge_cache/emshare/original/public/users/202103/f1/092cf1840/Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2020.xlsx'
SHEET_CRIME_VIC_NAME = 'Table 03'
victoria_crime_data = pd.read_excel(CRIME_VIC_URL,sheet_name=SHEET_CRIME_VIC_NAME)
victoria_crime_data.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2020,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1
1,2020,December,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B41 Motor vehicle theft,1
2,2020,December,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
3,2020,December,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
4,2020,December,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1


In [24]:
# for consistency, change the selected column name into Suburb
victoria_crime_data.rename(columns={'Suburb/Town Name':'Suburb'}, inplace=True)
victoria_crime_data.head()

,Year,Year ending,Local Government Area,Postcode,Suburb,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2020,December,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1
1,2020,December,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B41 Motor vehicle theft,1
2,2020,December,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
3,2020,December,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
4,2020,December,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1


In [25]:
victoria_crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317503 entries, 0 to 317502
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Year                   317503 non-null  int64 
 1   Year ending            317503 non-null  object
 2   Local Government Area  317503 non-null  object
 3   Postcode               317503 non-null  int64 
 4   Suburb                 317503 non-null  object
 5   Offence Division       317503 non-null  object
 6   Offence Subdivision    317503 non-null  object
 7   Offence Subgroup       317503 non-null  object
 8   Incidents Recorded     317503 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 21.8+ MB


For consistency, change the suburb name with `Postcode == 3004` from `Melbourne` to `St Kilda Rd`

In [26]:
victoria_crime_data.loc[victoria_crime_data['Postcode'] == 3004, 'Suburb'] = 'St Kilda Rd'
victoria_crime_data.loc[victoria_crime_data['Postcode'] == 3004].head()

,Year,Year ending,Local Government Area,Postcode,Suburb,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
20543,2020,December,Melbourne,3004,St Kilda Rd,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,3
20544,2020,December,Melbourne,3004,St Kilda Rd,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,14
20545,2020,December,Melbourne,3004,St Kilda Rd,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",12
20546,2020,December,Melbourne,3004,St Kilda Rd,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,6
20547,2020,December,Melbourne,3004,St Kilda Rd,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,13


Merge the Victoria Crime data with the prepared geographical data of Metro Melbourne so that the crime incident data of Metro Melbourne can be obtained

In [27]:
melbourne_crime_data = victoria_crime_data.merge(melbourne_df_non_dup, how='right',left_on=['Suburb','Postcode'], right_on=['Suburb','Postcode'])
melbourne_crime_data.head(10)

,Year,Year ending,Local Government Area,Postcode,Suburb,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Council,Latitude,Longitude
0,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,90.0,City of Melbourne,-37.8142176,144.9631608
1,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,349.0,City of Melbourne,-37.8142176,144.9631608
2,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",102.0,City of Melbourne,-37.8142176,144.9631608
3,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,136.0,City of Melbourne,-37.8142176,144.9631608
4,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,482.0,City of Melbourne,-37.8142176,144.9631608
5,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A50 Robbery,A51 Aggravated robbery,82.0,City of Melbourne,-37.8142176,144.9631608
6,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,A50 Robbery,A52 Non-Aggravated robbery,8.0,City of Melbourne,-37.8142176,144.9631608
7,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A711 FV Stalking,22.0,City of Melbourne,-37.8142176,144.9631608
8,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A712 Non-FV Stalking,30.0,City of Melbourne,-37.8142176,144.9631608
9,2020.0,December,Melbourne,3000,Melbourne,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A721 FV Harassment and private nuisance,10.0,City of Melbourne,-37.8142176,144.9631608


In [28]:
melbourne_crime_data.shape

(170541, 12)

In [29]:
melbourne_crime_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170541 entries, 0 to 170540
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Year                   170516 non-null  float64
 1   Year ending            170516 non-null  object 
 2   Local Government Area  170516 non-null  object 
 3   Postcode               170541 non-null  int64  
 4   Suburb                 170541 non-null  object 
 5   Offence Division       170516 non-null  object 
 6   Offence Subdivision    170516 non-null  object 
 7   Offence Subgroup       170516 non-null  object 
 8   Incidents Recorded     170516 non-null  float64
 9   Council                170541 non-null  object 
 10  Latitude               170541 non-null  object 
 11  Longitude              170541 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 16.9+ MB


From the above, it can be seen that some suburbs have no crime record found in the dataset. The following code is to check both datasets to ensure the consistency and try to eliminate mismatches, reducing the number of NaN in the dataset. 

In [30]:
# create a list containing suburb name and postcode that has no crime incident records.
missing_list = melbourne_crime_data[melbourne_crime_data['Incidents Recorded'].isnull()][['Suburb','Postcode']]
missing_list = missing_list.reset_index()
missing_list

,index,Suburb,Postcode
0,17256,Manor Lakes,3024
1,23908,"Flemington, Victoria",3031
2,65907,Bend of Islands,3097
3,135312,Quandong,3340
4,138853,Woodstock,3755
5,142166,Kinglake West,3778
6,146162,Warburton East,3799
7,146163,Monash University,3800
8,150557,Dewhurst,3809
9,153059,Koo Wee Rup North,3891


With few searches on google, it can be found that, in geographical dataset, some suburbs placed in the list have been given the `wrong postcodes`. The follow code is to correct the wrong postcode using the correct postcode provided in the Victoria Crime dataset. 

Furthermore, a suburb named `Flemington, Victoria` was found in geographical dataset. It is a duplicate of the suburb named `Flemington` which should be removed from the geographical dataset. 

At last, there is no crime data found for the suburb named `Monash University`. After check postcode==3800 in the Victoria Crime dataset, it was found that the suburb was labeled as `Clayton` in Victoria Crime dataset. To maintain consistency with geographical dataset, the suburb's name was changed to `Monash University`.

In [31]:
# two lists named missing_list and rest_list will be created
wrong_postcode = []
rest_list = []
for i in range(missing_list.shape[0]):
    if len(victoria_crime_data[victoria_crime_data['Suburb'] == missing_list.iloc[i,1]]):
        wrong_postcode.append(missing_list.iloc[i,1])
    else:
        if missing_list.iloc[i,1] == 'Monash University' or missing_list.iloc[i,1] == 'Flemington, Victoria':
            continue
        rest_list.append(missing_list.iloc[i,1])
print(wrong_postcode,'\n\n',rest_list)

['Quandong', 'Woodstock', 'Kinglake West', 'Dewhurst', 'Koo Wee Rup North', 'Lang Lang East', 'Yannathan'] 

 ['Manor Lakes', 'Bend of Islands', 'Warburton East', 'HMAS Cerberus', 'Capel Sound', 'Tooradin North', 'Fraser Rise', 'Deanside', 'Fieldstone', 'Grangefields', 'Harkness', 'Strathtulloh', 'Weir Views', 'Thornhill Park', 'Cobblebank', 'Aintree']


In [32]:
melbourne_df_non_dup.shape

(546, 5)

In [33]:
# correct the postcode for selected suburbs
postcode_corrections = []
multiple_postcode = []
for name in wrong_postcode:
    postcode = victoria_crime_data[victoria_crime_data['Suburb'] == name]['Postcode'].value_counts().index
    if len(postcode) == 1:
        print(melbourne_df_non_dup[melbourne_df_non_dup['Suburb'] == name][['Suburb','Postcode']])
        melbourne_df_non_dup.loc[melbourne_df_non_dup['Suburb'] == name,'Postcode'] = postcode
        print(melbourne_df_non_dup[melbourne_df_non_dup['Suburb'] == name][['Suburb','Postcode']])
        
        
        postcode_corrections.append([name,postcode[0]])
    else:
        multiple_postcode.append([name,postcode])
if multiple_postcode == []:
    print('\n')
    print('Not Found multiple postcodes in any suburb!')
print('Correction on melbourne_df_non_dup is completed!')
postcode_corrections

       Suburb  Postcode
352  Quandong      3340
       Suburb  Postcode
352  Quandong      3030
        Suburb  Postcode
364  Woodstock      3755
        Suburb  Postcode
364  Woodstock      3751
            Suburb  Postcode
389  Kinglake West      3778
            Suburb  Postcode
389  Kinglake West      3757
       Suburb  Postcode
436  Dewhurst      3809
       Suburb  Postcode
436  Dewhurst      3808
                Suburb  Postcode
457  Koo Wee Rup North      3891
                Suburb  Postcode
457  Koo Wee Rup North      3810
             Suburb  Postcode
458  Lang Lang East      3894
             Suburb  Postcode
458  Lang Lang East      3984
        Suburb  Postcode
459  Yannathan      3894
        Suburb  Postcode
459  Yannathan      3981


Not Found multiple postcodes in any suburb!
Correction on melbourne_df_non_dup is completed!


[['Quandong', 3030],
 ['Woodstock', 3751],
 ['Kinglake West', 3757],
 ['Dewhurst', 3808],
 ['Koo Wee Rup North', 3810],
 ['Lang Lang East', 3984],
 ['Yannathan', 3981]]

In [34]:
melbourne_df_non_dup.shape

(546, 5)

In [35]:
# check the results
postcode_corrections_df = pd.DataFrame(postcode_corrections,columns= ['Suburb','Postcode'])
postcode_corrections_df

,Suburb,Postcode
0,Quandong,3030
1,Woodstock,3751
2,Kinglake West,3757
3,Dewhurst,3808
4,Koo Wee Rup North,3810
5,Lang Lang East,3984
6,Yannathan,3981


In [36]:
# suburbs with no crime record exists
melbourne_df_non_dup.merge(postcode_corrections_df,how='inner')

,Suburb,Postcode,Council,Latitude,Longitude
0,Quandong,3030,City of Wyndham,-37.8388844,144.5538125
1,Woodstock,3751,City of Whittlesea,-37.54608,145.05437385869567
2,Kinglake West,3757,Shire of Nillumbik,-37.478028,145.25571422949724
3,Dewhurst,3808,Shire of Cardinia,-37.9761335,145.47387508579544
4,Koo Wee Rup North,3810,Shire of Cardinia,-38.145267000000004,145.5371992735536
5,Lang Lang East,3984,Shire of Cardinia,-38.270215,145.65643605931228
6,Yannathan,3981,Shire of Cardinia,-38.2242665,145.65067561205592


In [37]:
melbourne_df_non_dup.shape

(546, 5)

In [38]:
# remove Flemington, Victoria from geographical dataset
melbourne_df_non_dup.drop(melbourne_df_non_dup[melbourne_df_non_dup['Suburb'] == 'Flemington, Victoria'].index, inplace=True)

In [39]:
# change suburb(Postcode==3800)'s name from Clayton to Monash University
victoria_crime_data.loc[victoria_crime_data['Postcode'] == 3800, 'Suburb'] = 'Monash University'
victoria_crime_data[victoria_crime_data['Postcode'] == 3800].head()

,Year,Year ending,Local Government Area,Postcode,Suburb,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
23601,2020,December,Monash,3800,Monash University,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,2
23602,2020,December,Monash,3800,Monash University,A Crimes against the person,A50 Robbery,A51 Aggravated robbery,1
23603,2020,December,Monash,3800,Monash University,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A712 Non-FV Stalking,1
23604,2020,December,Monash,3800,Monash University,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A722 Non-FV Harassment and private nuisance,2
23605,2020,December,Monash,3800,Monash University,A Crimes against the person,Other crimes against the person,Other crimes against the person,5


In [40]:
# merge two dataset again to get crime records for Metro Melbourne
melbourne_crime_data = victoria_crime_data.merge(melbourne_df_non_dup, how='right',left_on=['Suburb','Postcode'], right_on=['Suburb','Postcode'])
melbourne_crime_data.shape

(171120, 12)

In [41]:
# examine the remain suburbs with empty crime records.
melbourne_crime_data[melbourne_crime_data['Incidents Recorded'].isnull()][['Suburb','Postcode']].reset_index()

,index,Suburb,Postcode
0,17256,Manor Lakes,3024
1,65906,Bend of Islands,3097
2,146433,Warburton East,3799
3,156995,HMAS Cerberus,3920
4,161232,Capel Sound,3940
5,166687,Tooradin North,3977
6,170750,Fraser Rise,3336
7,170751,Deanside,3336
8,171112,Fieldstone,3024
9,171113,Grangefields,3335


From the above results, it can be seen that there are still 15 suburbs without having crime records. By checking them on Google Maps individually, it is safe to say that these suburbs are not suitable for holding desired business activities since they are far from the central areas of Melbourne. Therefore, the suburbs having empty crime records will be ignored in this case. 

#### Rental Dataset of Metropolitan Melbourne

In this section, the rental dataset of Metro Melbourne is extracted from the Victoria Rental Report. The reference link is shown below.

https://www.dhhs.vic.gov.au/sites/default/files/documents/202103/Moving%20annual%20rent%20by%20suburb%20-%20December%20quarter%202020.xlsx

In [42]:
# download the raw rental data
RENTAL_VIC_URL = 'https://www.dhhs.vic.gov.au/sites/default/files/documents/202103/Moving%20annual%20rent%20by%20suburb%20-%20December%20quarter%202020.xlsx'
SHEET_RENTAL_VIC_NAME = 'All properties'
victoria_rental_raw = pd.read_excel(RENTAL_VIC_URL,sheet_name=SHEET_RENTAL_VIC_NAME,header=[1,2])

In [43]:
victoria_rental_raw.head()

All properties                                        Mar 2000         \
  Unnamed: 0_level_1                     Unnamed: 1_level_1    Count Median   
0    Inner Melbourne  Albert Park-Middle Park-West St Kilda     1146    260   
1                NaN                               Armadale      729    200   
2                NaN                          Carlton North      864    260   
3                NaN                      Carlton-Parkville     1344    260   
4                NaN                        CBD-St Kilda Rd     2138    320   

  Jun 2000        Sep 2000        Dec 2000         ... Dec 2019         \
     Count Median    Count Median    Count Median  ...    Count Median   
0     1136    260     1177    270     1176    275  ...      848    595   
1      731    200      732    205      732    210  ...      776    485   
2      814    260      800    265      735    270  ...      520    580   
3     1312    260     1306    260     1322    260  ...     4606    420   
4     2269    320     2362    320     2361    320  ...    10840    510   

  Mar 2020        Jun 2020        Sep 2020        Dec 2020         
     Count Median    Count Median    Count Median    Count Median  
0      852    600      826    585      798    560      829    550  
1      753    500      782    500      708    500      715    495  
2      539    585      536    590      519    590      536    590  
3     5211    404     5023    400     4625    400     4655    399  
4    10896    510    11022    495    11295    460    13146    420  

[5 rows x 170 columns]

In [44]:
# check the shape of the dataset
victoria_rental_raw.shape

(159, 170)

In [45]:
# re-organize the dataset and clarify the column names
victoria_rental_data = pd.Series(victoria_rental_raw.iloc[:,1],name='Suburb')
year = 2000

for i in range(2,victoria_rental_raw.shape[1]-7,8):

    col_name = ['Count_Mar_{}'.format(year),
                    'Median_Mar_{}'.format(year),
                    'Count_Jun_{}'.format(year),
                    'Median_Jun_{}'.format(year),
                    'Count_Sep_{}'.format(year),
                    'Median_Sep_{}'.format(year),
                    'Count_Dec_{}'.format(year),
                    'Median_Dec_{}'.format(year),]
    temp_df = pd.DataFrame(np.array(victoria_rental_raw.iloc[:,i:i+8]),columns=col_name)
    victoria_rental_data = pd.concat([victoria_rental_data, temp_df],axis=1)
    year +=1
    
victoria_rental_data.head()

,Suburb,Count_Mar_2000,Median_Mar_2000,Count_Jun_2000,Median_Jun_2000,Count_Sep_2000,Median_Sep_2000,Count_Dec_2000,Median_Dec_2000,Count_Mar_2001,...,Count_Dec_2019,Median_Dec_2019,Count_Mar_2020,Median_Mar_2020,Count_Jun_2020,Median_Jun_2020,Count_Sep_2020,Median_Sep_2020,Count_Dec_2020,Median_Dec_2020
0,Albert Park-Middle Park-West St Kilda,1146,260,1136,260,1177,270,1176,275,1206,...,848,595,852,600,826,585,798,560,829,550
1,Armadale,729,200,731,200,732,205,732,210,716,...,776,485,753,500,782,500,708,500,715,495
2,Carlton North,864,260,814,260,800,265,735,270,715,...,520,580,539,585,536,590,519,590,536,590
3,Carlton-Parkville,1344,260,1312,260,1306,260,1322,260,1272,...,4606,420,5211,404,5023,400,4625,400,4655,399
4,CBD-St Kilda Rd,2138,320,2269,320,2362,320,2361,320,2587,...,10840,510,10896,510,11022,495,11295,460,13146,420


In [46]:
victoria_rental_data.shape

(159, 169)

In [47]:
# delete rows called Group Total in the dataset
victoria_rental_data = victoria_rental_data[victoria_rental_data['Suburb'] != 'Group Total']
victoria_rental_data.shape

(146, 169)

From the preliminary examination, it can be found that several small suburbs were combined together to represent the rental status in those local areas. To maintain the consistency with two prepared datasets above, the combined suburbs will be seperated into individual suburb. The rental counts for the local area will be divided evenly among each suburb. However, the median price will remain no change. 

In [48]:
# define a function to split the combine suburbs and evenly divide their rental counts

def splitsuburbs(df):
    # create a new dataframe
    new_df = pd.DataFrame(columns=df.columns)
    
    # split the name
    for i in range(df.shape[0]):
        temp_name = re.split(r'-',df.iloc[i,0])
        rest_data = np.array(df.iloc[i,1:])
        
        # if it is combined suburb
        if len(temp_name)>=2:
            rest_data[::2] //= len(temp_name)
            for suburb in temp_name:
                new_data =  pd.DataFrame(np.insert(rest_data,0,suburb).reshape(1,-1),columns=df.columns)
                new_df = new_df.append(new_data)
        
        # if it is a single suburb
        else:
            temp_name = temp_name[0]
            new_data =  pd.DataFrame(np.insert(rest_data,0,temp_name).reshape(1,-1),columns=df.columns)
            new_df = new_df.append(new_data)

    return new_df

In [49]:
victoria_rental_data_processed = splitsuburbs(victoria_rental_data)

In [50]:
victoria_rental_data_processed.shape

(215, 169)

To ensure consistency, few suburbs' names found in rental data need to be changed.

In [51]:
victoria_rental_data_processed.replace('CBD', 'Melbourne', inplace=True)
victoria_rental_data_processed.replace('West St Kilda', 'St Kilda West', inplace=True)
victoria_rental_data_processed.replace('East St Kilda', 'St Kilda East', inplace=True)

In [52]:
# merge with geographical data of metro melbourne to obtain the rental data for metro melbourne
melbourne_rental_data = victoria_rental_data_processed.merge(melbourne_df_non_dup,how='right',on=['Suburb'])
melbourne_rental_data.head()

,Suburb,Count_Mar_2000,Median_Mar_2000,Count_Jun_2000,Median_Jun_2000,Count_Sep_2000,Median_Sep_2000,Count_Dec_2000,Median_Dec_2000,Count_Mar_2001,...,Count_Jun_2020,Median_Jun_2020,Count_Sep_2020,Median_Sep_2020,Count_Dec_2020,Median_Dec_2020,Postcode,Council,Latitude,Longitude
0,Melbourne,1069,320,1134.0,320.0,1181,320,1180,320,1293,...,5511.0,495.0,5647.0,460.0,6573.0,420.0,3000,City of Melbourne,-37.8142176,144.9631608
1,East Melbourne,495,265,490.0,268.0,516,280,506,273,494,...,629.0,550.0,562.0,515.0,544.0,495.0,3002,City of Melbourne,-37.815588,144.98255534496568
2,West Melbourne,468,230,478.0,240.0,491,240,497,242,521,...,1741.0,455.0,1637.0,450.0,1665.0,425.0,3003,City of Melbourne,-37.8106885,144.9204039725214
3,St Kilda Rd,1069,320,1134.0,320.0,1181,320,1180,320,1293,...,5511.0,495.0,5647.0,460.0,6573.0,420.0,3004,City of Melbourne,-37.8358215,144.9763639
4,South Wharf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3006,City of Port Phillip,-37.8253445,144.9523945305609


In [53]:
melbourne_rental_data.shape

(545, 173)

In [54]:
# number of suburbs without rental data
melbourne_rental_data[melbourne_rental_data.iloc[:,1].isnull()]['Suburb'].count()

383

#### Venues in Metro Melbourne

In this section, the veunes of each suburb in Metro Melbourne were collected using Foursquare API. Since a free Foursquare dev. account was used in this project which only allows to make 100-item-per-API-call. Unfortunately, the maximum number of venues in each suburb was capped at 100 venues.

In [55]:
# define Foursquare Credentials and Version
CLIENT_ID = 'LEDK5OOKF4IRDV30JKIFZ0B1T5T3CRTN1PAFIEDI3XUACT5C' # your Foursquare ID
CLIENT_SECRET = 'DEK45GQV3YIOEKRFKYZ04FAVVYSQ1MCZFF1L54UAVKNJUZ3I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LEDK5OOKF4IRDV30JKIFZ0B1T5T3CRTN1PAFIEDI3XUACT5C
CLIENT_SECRET:DEK45GQV3YIOEKRFKYZ04FAVVYSQ1MCZFF1L54UAVKNJUZ3I


In [56]:
# define a function to call Foursquare API and extracting venue information in given coordinates
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
# create a dataframe for venues in Metro Melbourne
melbourne_venue_data = getNearbyVenues(melbourne_df_non_dup['Suburb'],
                                       melbourne_df_non_dup['Latitude'],
                                       melbourne_df_non_dup['Longitude'])

Melbourne
East Melbourne
West Melbourne
St Kilda Rd
South Wharf
Southbank
Docklands
Seddon
Footscray
Kingsville
West Footscray
Brooklyn
Tottenham
Maidstone
Yarraville
South Kingsville
Newport
Spotswood
Williamstown North
Williamstown
Altona
Seaholme
Braybrook
Albion
Sunshine
Sunshine North
Sunshine West
Kings Park
St Albans
Albanvale
Kealba
Ardeer
Deer Park
Cairnlea
Burnside
Burnside Heights
Caroline Springs
Ravenhall
Mount Cottrell
Mambourin
Wyndham Vale
Manor Lakes
Altona North
Laverton North
Williams Landing
Seabrook
Laverton
Altona Meadows
Hoppers Crossing
Tarneit
Truganina
Point Cook
Derrimut
Cocoroc
Werribee
Werribee South
Flemington
Kensington
Ascot Vale
Travancore
Maribyrnong
Keilor East
Avondale Heights
Keilor North
Keilor
Hillside
Calder Park
Sydenham
Delahey
Taylors Lakes
Keilor Downs
Keilor Lodge
Moonee Ponds
Aberfeldie
Essendon
Essendon West
Strathmore Heights
Strathmore
Essendon Fields
Essendon North
Airport West
Keilor Park
Niddrie
Tullamarine
Gowanbrae
Gladstone Park
Pa

In [58]:
melbourne_venue_data.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne,-37.8142176,144.9631608,Tipo 00,-37.813527,144.961978,Italian Restaurant
1,Melbourne,-37.8142176,144.9631608,Brother Baba Budan,-37.813445,144.962137,Coffee Shop
2,Melbourne,-37.8142176,144.9631608,Trattoria Emilia,-37.815220,144.962636,Italian Restaurant
3,Melbourne,-37.8142176,144.9631608,Kirk's Wine Bar,-37.813661,144.961351,Wine Bar
4,Melbourne,-37.8142176,144.9631608,Whitehart,-37.812781,144.961973,Bar


In [59]:
melbourne_venue_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3966 entries, 0 to 3965
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Suburb            3966 non-null   object 
 1   Suburb Latitude   3966 non-null   object 
 2   Suburb Longitude  3966 non-null   object 
 3   Venue             3966 non-null   object 
 4   Venue Latitude    3966 non-null   float64
 5   Venue Longitude   3966 non-null   float64
 6   Venue Category    3966 non-null   object 
dtypes: float64(2), object(5)
memory usage: 217.0+ KB


#### Saving Datesets into CSV files

In this section, the prepared datasets were saved as .csv in datasets folder 

In [60]:
def save_dataset(filename):
    return os.path.join(DATASETS_PATH,filename)

In [61]:
# save prepared datasets
melbourne_df_non_dup.to_csv(save_dataset('geo_data_mel'))
melbourne_crime_data.to_csv(save_dataset('crime_data_mel'))
melbourne_rental_data.to_csv(save_dataset('rental_data_mel'))
melbourne_venue_data.to_csv(save_dataset('venue_data_mel'))